# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [26]:
reset_kernel = True

In [1]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Config parameters
> Configuration parameters are obtained from 'config\03-embeddings.yaml'

### Get configuration artifact

In [2]:
config, job_type = get_artifact_config_embeddings(verbose = 0)

In [3]:
dvats.config.show_attrdict(config)

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/mvp-SWV:latest
input_ar: None
cpu: False


### Show configuration artifact

In [4]:
for key, value in config.items():
    print(f"{key}: {value}")

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/mvp-SWV:latest
input_ar: None
cpu: False


## Build W&B artifact

In [5]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="03a_embeddings"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
print("runname: "+runname)

runname: 03a_embeddings


In [6]:
run = wandb.init(
    entity      = config.wandb_entity,
    project     = config.wandb_project if config.use_wandb else 'work-nbs', 
    group       = config.wandb_group,
    job_type    = job_type,
    mode        = 'online' if config.use_wandb else 'disabled',
    anonymous   = 'never' if config.use_wandb else 'must',
    config      = config,
    resume      = 'allow',
    name        = runname
)

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


'stream.Stream' object attribute 'write' is read-only


## Get trained model artifact

### Build artifact selector
> Botch to use artifacts offline

In [7]:
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

### Get the model from W&B
> Restore the encoder model and its associated configuration

In [8]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [9]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

wandb:   1 of 1 files downloaded.  


## Get dataset artifact from W&B
### Restore the dataset artifact used for training the encoder. 
> Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that it matches with the dataset that we want to reduce.

In [10]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

'Monash-Australian_electricity_demand:v7'

In [11]:
dvats.config.show_attrdict(enc_run.config)

r: 0.71
w: 30
MVP: {'r': 0.71, 'lm': 3, 'crit': None, 'sync': False, 'fname': 'encoder_MVP', 'dropout': 0.1, 'verbose': False, 'stateful': True, 'save_best': True, 'nan_to_num': 0, 'custom_mask': None, 'future_mask': False, 'weights_path': None, 'variable_mask': False, 'subsequence_mask': True}
alias: Monash-Australian_electricity_demand
n_inp: 1
device: cuda
epochs: 100
frozen: False
mvp_ws: [15, 30]
stride: 15
Learner: {'lr': 0.001, 'wd': None, 'arch': 'tsai.models.InceptionTimePlus.InceptionTimePlus', 'moms': [0.95, 0.85, 0.95], 'path': '.', '_name': '<fastai.learner.Learner object at 0x7f829251fe50>', 'metrics': None, 'opt_func': 'fastai.optimizer.Adam', 'splitter': 'tsai.models.utils.ts_splitter', 'train_bn': True, 'loss_func': {'axis': -1, '_name': {'axis': -1, '_name': 'FlattenedLoss of MSELoss()', 'is_2d': False, 'flatten': True, 'floatify': True}, 'is_2d': False, 'flatten': True, 'floatify': True}, 'model_dir': 'models', 'wd_bn_bias': False, 'default_cbs': True}
Recorder: {'ad

### Specify the dataset artifact that we want to get the embeddings from
> If no artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [12]:
enc_run.config['batch_size']

512

In [13]:
input_ar_name = ifnone(
    config.input_ar, 
    f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}'
)
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'Monash-Australian_electricity_demand:v7'

In [14]:
df = input_ar.to_df()
df.head()

wandb:   1 of 1 files downloaded.  


,penguin_sample
1970-01-01 00:00:00,0.253906
1970-01-01 10:00:00,0.259033
1970-01-01 20:00:00,0.269287
1970-01-02 06:00:00,0.271240
1970-01-02 16:00:00,0.265137


In [15]:
df.shape

(109842, 1)

In [16]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

(7321, 1, 30)

In [17]:
enc_learner.dls.bs

1

In [18]:
enc_learner.dls

In [19]:
import dvats.utils as ut

In [20]:
timer = ut.Time()
timer.start()

1725374080.422395

In [21]:
embs = get_enc_embs(
    X          = enc_input, 
    enc_learn  = enc_learner, 
    cpu        = config.cpu, 
    to_numpy   = True,
    batch_size = enc_run.config['batch_size'],
    verbose    = 1
)

[ Get Encoder Embeddings ] --> GPU
[ Get Encoder Embeddings ] GPU -->
[ Get Encoder Embeddings ] Set dataloader from X (enc_learn does not contain dls)
[ Get Encoder Embeddings ] Get module
[ Get Encoder Embeddings ] get_acts_and_grads 
[ Get Encoder Embeddings ] get_acts_and_grads | --> Concat
[ Get Encoder Embeddings ] get_acts_and_grads | Concat -->
[ Get Encoder Embeddings ] Reduce to 2 dimensions.
[ Get Encoder Embeddings ] Ensure CPU saving & numpy format


In [22]:
timer.end()
timer.show()

[] Start: 1725374080.422395 | End: 1725374081.9342782 | Duration: 1.51188325881958 seconds


1.51188325881958

In [23]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings', metadata=dict(run.config)), 
                     aliases=f'run-{run.project}-{run.id}')

In [24]:
run.finish()

In [ ]:
#| export
print("Execution ended")
beep(1)
if reset_kernel:
    import os
    os._exit(00)

Execution ended
